# Developing text objects further

In [1]:
import sys; sys.path.insert(0,'/Users/ryan/github/cadence/')
from cadence.imports import *

In [330]:
s='Turning and turning in the widening gyre'
txt="""
Turning and turning in the widening gyre  , the falcon cannot hear the falconer.
Things fall apart; the centre cannot hold.

Mere anarchy is loosed upon the world. 
The blood-dimmed tide is loosed, and everywhere the ceremony of innocence is drowned;
The best lack all conviction, while the worst   
Are full of passionate intensity.
"""

In [457]:
s='''The best lack all conviction,
while the worst Are full of passionate intensity.'''

In [460]:
import stanza
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,lemma,depparse,constituency')
doc = nlp(s)
sent = doc.sentences[0]
tree = sentence.constituency
s

2022-01-06 20:52:07 INFO: Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| lemma        | combined |
| depparse     | combined |
| constituency | wsj      |

INFO:stanza:Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| lemma        | combined |
| depparse     | combined |
| constituency | wsj      |

2022-01-06 20:52:07 INFO: Use device: cpu
INFO:stanza:Use device: cpu
2022-01-06 20:52:07 INFO: Loading: tokenize
INFO:stanza:Loading: tokenize
2022-01-06 20:52:07 INFO: Loading: pos
INFO:stanza:Loading: pos
2022-01-06 20:52:07 INFO: Loading: lemma
INFO:stanza:Loading: lemma
2022-01-06 20:52:07 INFO: Loading: depparse
INFO:stanza:Loading: depparse
2022-01-06 20:52:08 INFO: Loading: constituency
INFO:stanza:Loading: constituency
2022-01-06 20:52:08 INFO: Done loading

'The best lack all conviction,\nwhile the worst Are full of passionate intensity.'

In [461]:
NLPD={}

def get_nlp(**kwargs):
    global NLPD
    key=kwargs_key(kwargs)
    if not key in NLPD:
        import stanza
        NLPD[key] = stanza.Pipeline(**kwargs)
    return NLPD[key]


In [462]:


def recurse_tree(tree,node_i=0,path=[]):
    o=[]
    if not hasattr(tree,'node_i'):
        tree.node_i=node_i
        node_i+=1
    if not tree.is_leaf():
        path+=[f'{tree.node_i}-{tree.label}']
        for x in tree.children:
            o+=recurse_tree(x,node_i=node_i,path=list(path))
    else:
        o+=[tuple(path)]
        path=[]
    return o


def tokenize_nlp_iter(txt,lang=DEFAULT_LANG,engine='',sep_line=SEP_LINE, sep_para=SEP_STANZA, seps_phrase=SEPS_PHRASE):
    proc='tokenize,pos,lemma,depparse,constituency'
    badcols={'feats','start_char','end_char','id'}
    nlp=get_nlp(lang=lang, processors=proc)
    
    char_i=0
    para_i=1
    line_i=1
    linepart_i=1
    linepart_ii=0
        
    doc=nlp(txt)
    for sent_i, sent in enumerate(doc.sentences):
        senttree = recurse_tree(sent.constituency, node_i=0, path=[])
        for tok_i,tok in enumerate(sent.tokens):
            prefstr=txt[char_i:tok.start_char]
            tokstr=txt[tok.start_char : tok.end_char]
            realtok=txt[char_i:tok.end_char]
            if sep_para in prefstr: para_i+=1
            if sep_line in prefstr: line_i+=1
            char_i=tok.end_char
            tokd=tok.to_dict()[0]
            word_i=tok_i+1
            assert tok.id[0]==word_i
            
            odx_tokd=dict((k,v) for k,v in tokd.items() if k not in badcols)
            
            odx_feat={}
            for feat in tokd.get('feats','').split('|'):
                if not feat: continue
                fk,fv=feat.split('=',1)
                odx_feat[fk]=fv
                
            odx_const=dict(
                depth=len(senttree[tok_i]),
                constituency='('+'('.join(senttree[tok_i])+'('
            )
            
            odx=dict(
                para_i=para_i,
                sent_i=sent_i+1,
                linepart_i=linepart_i,
                line_i=line_i,
                word_i=tok_i+1,
                word_pref=prefstr,
                word=tokstr,
                **odx_tokd,
                **odx_const,
                **odx_feat,
            )
            
            
            yield odx
            
            if set(realtok)&set(seps_phrase): linepart_i+=1

def tokenize_nlp(txt,**kwargs):
    odf=pd.DataFrame(tokenize_nlp_iter(txt)).fillna('')
    return odf
    

In [463]:
tokenize_nlp(s)

2022-01-06 20:52:13 INFO: Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| lemma        | combined |
| depparse     | combined |
| constituency | wsj      |

INFO:stanza:Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| lemma        | combined |
| depparse     | combined |
| constituency | wsj      |

2022-01-06 20:52:13 INFO: Use device: cpu
INFO:stanza:Use device: cpu
2022-01-06 20:52:13 INFO: Loading: tokenize
INFO:stanza:Loading: tokenize
2022-01-06 20:52:13 INFO: Loading: pos
INFO:stanza:Loading: pos
2022-01-06 20:52:13 INFO: Loading: lemma
INFO:stanza:Loading: lemma
2022-01-06 20:52:13 INFO: Loading: depparse
INFO:stanza:Loading: depparse
2022-01-06 20:52:13 INFO: Loading: constituency
INFO:stanza:Loading: constituency
2022-01-06 20:52:13 INFO: Done loading

,para_i,sent_i,linepart_i,line_i,word_i,word_pref,word,text,lemma,upos,xpos,head,deprel,depth,constituency,Definite,PronType,Degree,Number,Mood,Tense,VerbForm
0,1,1,1,1,1,,The,The,the,DET,DT,3,det,4,(0-ROOT(1-S(2-NP(3-DT(,Def,Art,,,,,
1,1,1,1,1,2,,best,best,good,ADJ,JJS,3,amod,4,(0-ROOT(1-S(2-NP(3-JJS(,,,Sup,,,,
2,1,1,1,1,3,,lack,lack,lack,NOUN,NN,5,nsubj,4,(0-ROOT(1-S(2-NP(3-NN(,,,,Sing,,,
3,1,1,1,1,4,,all,all,all,DET,DT,5,det,4,(0-ROOT(1-S(2-NP(3-DT(,,,,,,,
4,1,1,1,1,5,,conviction,conviction,conviction,NOUN,NN,0,root,4,(0-ROOT(1-S(2-NP(3-NN(,,,,Sing,,,
5,1,1,1,1,6,,",",",",",",PUNCT,",",11,punct,3,"(0-ROOT(1-S(2-,(",,,,,,,
6,1,1,2,2,7,\n,while,while,while,SCONJ,IN,11,mark,4,(0-ROOT(1-S(2-SBAR(3-IN(,,,,,,,
7,1,1,2,2,8,,the,the,the,DET,DT,9,det,6,(0-ROOT(1-S(2-SBAR(3-S(4-NP(5-DT(,Def,Art,,,,,
8,1,1,2,2,9,,worst,worst,bad,ADJ,JJS,11,nsubj,6,(0-ROOT(1-S(2-SBAR(3-S(4-NP(5-JJS(,,,Sup,,,,
9,1,1,2,2,10,,Are,Are,be,AUX,VBP,11,cop,6,(0-ROOT(1-S(2-SBAR(3-S(4-VP(5-VBP(,,,,,Ind,Pres,Fin


### Metrical tree?

In [464]:
def get_dtree_str(sent):
    o=[]
    for dep in sent.dependencies:
        w1,rel,w2=dep
        ostr=f'{rel}({w1.text}-{w1.id}, {w2.text}-{w2.id})'
        o+=[ostr]
    return '\n'.join(o)


In [465]:
from metricaltree import DependencyTree,MetricalTree

In [466]:
def get_ctree_str(sent):
    return str(sent.constituency)

In [468]:
ctree=get_ctree_str(sent)
ctree

'(ROOT (S (NP (DT The) (JJS best) (NN lack)) (NP (DT all) (NN conviction)) (, ,) (SBAR (IN while) (S (NP (DT the) (JJS worst)) (VP (VBP Are) (ADJP (JJ full) (PP (IN of) (NP (JJ passionate) (NN intensity))))))) (. .)))'

In [469]:
dtree=get_dtree_str(sent)
dtree

'det(lack-3, The-1)\namod(lack-3, best-2)\nnsubj(conviction-5, lack-3)\ndet(conviction-5, all-4)\nroot(ROOT-0, conviction-5)\npunct(full-11, ,-6)\nmark(full-11, while-7)\ndet(worst-9, the-8)\nnsubj(full-11, worst-9)\ncop(full-11, Are-10)\nadvcl(conviction-5, full-11)\ncase(intensity-14, of-12)\namod(intensity-14, passionate-13)\nobl(full-11, intensity-14)\npunct(conviction-5, .-15)'

In [470]:
bothtree=ctree+'\n\n'+dtree

In [471]:
DependencyTree.fromstring(bothtree)

AttributeError: module 'nltk.compat' has no attribute 'string_types'

In [4]:
nltk.sent_tokenize(txt)

['\nTurning and turning in the widening gyre.',
 'the falcon cannot hear the falconer;\nThings fall apart; the centre cannot hold;\nMere anarchy is loosed upon the world.',
 'The blood-dimmed tide is loosed, and everywhere the ceremony of innocence is drowned;\nThe best lack all conviction, while the worst   \nAre full of passionate intensity.']

In [5]:
to_lineparts_ld('hello world')

[{'stanza_i': 1, 'line_i': 1, 'linepart_i': 1, 'linepart_str': 'hello world '}]

In [6]:
# for odf in scan_iter(txt,max_len=2): display(odf)

In [7]:
scan(txt,max_len=2,linebreaks=True,phrasebreaks=True)

Scanning lines [x1]: 100%|███████████████████████████████████████████| 29/29 [00:00<00:00, 37.16it/s]


is_funcword  ...  prom_weight
stanza_i line_i linepart_i linepart_str           word_i word_str    word_ipa_i word_ipa       syll_i syll_str syll_ipa syll_stress syll_weight               ...             
1        1      1          Turning                1      Turning     1          'tɛː.nɪŋ       1      Tur      'tɛː     P                                  0  ...          NaN
                                                                                               2      ning     nɪŋ      U                                  0  ...          NaN
                2          and turning            1      and         1          ænd            1      and      ænd      U                                  1  ...          NaN
                                                  2      turning     1          'tɛː.nɪŋ       1      tur      'tɛː     P                                  0  ...          NaN
                                                                                               2      ning     nɪŋ      U                                  0  ...          NaN
...                                                                                                                                                      ...  ...          ...
                29         passionate intensity.  1      passionate  1          'pæ.ʃə.nət     3      nate     nət      U                                  0  ...          NaN
                                                  2      intensity.  1          ɪn.'tɛn.sə.tiː 1      in       ɪn       U                                  0  ...          NaN
                                                                                               2      ten      'tɛn     P                                  0  ...          NaN
                                                                                               3      si       sə       U                                  0  ...          NaN
                                                                                               4      ty.      tiː      U                                  0  ...          NaN

[93 rows x 12 columns]

In [8]:

def Verse(txt,**kwargs):
    kwargs={**dict(verse=True), **kwargs}
    return Text(txt,**kwargs)
def Poem(txt,**kwargs):
    kwargs={**dict(verse=True), **kwargs}
    return Text(txt,**kwargs)
def Prose(txt,**kwargs):
    kwargs={**dict(prose=True), **kwargs}
    return Text(txt,**kwargs)
def FreeVerse(txt,**kwargs):
    kwargs={**dict(linebreaks=True,phrasebreaks=True), **kwargs}
    return Text(txt,**kwargs)



# loading txt/strings
def to_fn_txt(txt_or_fn):
    # load txt
    if type(txt_or_fn)==str and not '\n' in txt_or_fn and os.path.exists(txt_or_fn):
        fn=txt_or_fn
        with open(fn,encoding='utf-8',errors='replace') as f:
            txt=f.read()
    else:
        fn=''
        txt=txt_or_fn
    return (fn,txt.strip())


### convenient objs
def kwargs_key(kwargs,bad_keys={'num_proc','progress','desc'}):
    return ', '.join(
        f'{k}={v}'
        for k,v in kwargs.items()
        if k not in bad_keys
    )
    
    
### texts
class Text(object):
    def __init__(self,txt_or_fn,**kwargs):
        
        self._scans={}
        self._parses={}
        self._lineparts={}
        self._num_lines=None
        self._num_stanzas=None
        self._infod={}
        self._kwargs=kwargs
        
        ## Load texts
        self.fn,self.txt=to_fn_txt(txt_or_fn)
        
    # def __repr__(self):
    #     o=self.txt.split('\n\n')[0] if self.txt is not None else ""
    #     o='\t' + '\n\t'.join(l for l in o.split('\n'))
    #     o=f'''<cadence.Text: {self.first_line} ({self.num_stanzas} stanza{"s" if self.num_stanzas>1 else ""}, {self.num_lines} line{"s" if self.num_lines>1 else ""})>'''.strip()
    #     #o='\n'.join(l.strip() for l in o.split('\n'))
    #     return o
    
    def kwargs(self,**kwargs):
        return {**self._kwargs, **kwargs}
    
    def get_kwargs_key(self,**kwargs):
        return kwargs_key(self.kwargs(**kwargs))


    
    ##################################################################
    ### Stanzas
    def stanzas(self,txt='',**kwargs):
        kwargs=self.kwargs(**kwargs)
        return to_stanzas_str(txt if txt else self.txt,**kwargs)    

    ##################################################################
    ### Lines
    
    def lines(self,txt='',linebreaks=False,prose=False,**kwargs):
        if not txt: txt=self.txt
        kwargs=self.kwargs(**kwargs)
        to_lines_now = to_lines_str if kwargs.get('linebreaks') or kwargs.get('verse') else to_sents_str
        return [
            l
            for stanza_str in self.stanzas(txt,**kwargs)
            for l in to_lines_now(stanza_str, **kwargs)
        ]
    
    def sentences(self,txt='',**kwargs):
        if not txt: txt=self.txt
        kwargs=self.kwargs(**kwargs)
        return [
            lp
            for line_str in self.lines(txt,**kwargs)
            for lp in to_sents_str(line_str, **kwargs)
        ]


    ##################################################################
    ### LINEPARTS
    
    ### Lineparts
    def lineparts(self, txt='', **kwargs):
        if not txt: txt=self.txt
        kwargs=self.kwargs(**kwargs)
        return [
            lp
            for line_str in self.lines(txt,**kwargs)
            for lp in to_lineparts_str(line_str, **kwargs)
        ]        
    
    ##################################################################
    ### SCANS
    
    def scan(self, force=False, **kwargs):
        key=self.get_kwargs_key(**kwargs)
        if force or not key in self._scans:
            self._scans[key]=scan(self.txt,**kwargs)
        return self._scans[key]

    
    
    ##################################################################
    ### PARSE
    
    def parse(self,
            force=False,
            verbose=True,
            only_best=False,
            only_unbounded=True,
            **kwargs):
        kwargs['verbose']=verbose
        kwargs_line={**self.kwargs, **kwargs, **{'by_syll':False}}
        kwargs_syll={**self.kwargs, **kwargs, **{'by_syll':True}}
        key_line=kwargs_key(kwargs_line)
        key_syll=kwargs_key(kwargs_syll)
        if force or not (key_syll in self._parses) or (not key_line in self._parses):
            self._parses[key_syll]=parse(self.txt, **kwargs_syll)
            self._parses[key_line]=to_lines(self._parses[key_syll])
        elif kwargs.get('verbose',True):
            for li,linedf in sorted(self._parses[key_syll].reset_index().groupby(['stanza_i','line_i'])):
                display(show_parse(linedf))

        self.infod=info_parses(self._parses[key_line])
        if verbose: printm(show_info_parses(self.infod))
                
        
    def parses(self,
            force=True,
            only_best=False,
            only_unbounded=True,
            **kwargs):
        kwargs={**self.kwargs, **kwargs}
        kwargs_line={**self.kwargs, **kwargs, **{'by_syll':False}}
        kwargs_syll={**self.kwargs, **kwargs, **{'by_syll':True}}
        key_line=kwargs_key(kwargs_line)
        key_syll=kwargs_key(kwargs_syll)
        key=key_syll if kwargs.get('by_syll') else key_line
        if not key in self._parses:
            self.parse(force=force, **kwargs)
        if not key in self._parses: return
        
        odf=self._parses[key]
        if only_unbounded and ('parse_is_bounded' in set(odf.index.names) or 'parse_is_bounded' in set(odf.columns)):
            #odf=odf[odf.parse_is_bounded==False]
            odf=odf.query('parse_is_bounded==False')
        if only_best and ('parse_rank' in set(odf.index.names) or 'parse_rank' in set(odf.columns)):
            odf=odf.query('parse_rank==1')
        
        return odf

    def best_parses(self, force=False, **kwargs):
        return self.parses(force=force,only_best=True,**kwargs)
    def all_parses(self, force=False,**kwargs):
        return self.parses(force=force,only_best=False,only_unbounded=False,**kwargs)
    def unbounded_parses(self, force=False,**kwargs):
        return self.parses(force=force,only_best=False,only_unbounded=True,**kwargs)

In [9]:
t=Text('saintsbury/txt/en.addison.prose.Saintsbury.psgs_quoted.txt', prose=True, test='hello')